In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,34676
2,Huelva,Confirmados PDIA 14 días,1150
3,Huelva,Tasa PDIA 14 días,"224,09727770524387"
4,Huelva,Confirmados PDIA 7 días,655
5,Huelva,Total Confirmados,34865
6,Huelva,Curados,32548
7,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  34676.0
/tmp/ipykernel_78105/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9508.0
/tmp/ipykernel_78105/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_78105/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_78105/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_78105/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 682 personas en los últimos 7 días 

Un positivo PCR cada 433 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,34676.0,655.0,1150.0,513170.0,127.638015,224.097278,102.0
Huelva-Costa,20120.0,459.0,793.0,289548.0,158.522939,273.875143,77.0
Huelva (capital),9508.0,211.0,332.0,143837.0,146.693827,230.816827,61.0
Condado-Campiña,11184.0,156.0,290.0,156231.0,99.852142,185.622572,21.0
Palos de la Frontera,761.0,21.0,28.0,11742.0,178.845171,238.460228,11.0
Lepe,2283.0,63.0,97.0,27880.0,225.968436,347.919656,6.0
Isla Cristina,2366.0,85.0,162.0,21393.0,397.326228,757.257047,5.0
Sierra de Huelva-Andévalo Central,3001.0,27.0,49.0,67391.0,40.064697,72.710006,5.0
Moguer,1464.0,27.0,46.0,21867.0,123.473728,210.362647,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Santa Bárbara de Casa,35.0,4.0,19.0,1043.0,383.509108,1821.668265,0.0
Niebla,297.0,31.0,48.0,4116.0,753.158406,1166.180758,1.0
Isla Cristina,2366.0,85.0,162.0,21393.0,397.326228,757.257047,5.0
Zarza-Perrunal (La),33.0,3.0,6.0,1253.0,239.425379,478.850758,0.0
Alosno,207.0,5.0,18.0,3933.0,127.129418,457.665904,0.0
Puerto Moral,5.0,1.0,1.0,278.0,359.712230,359.712230,0.0
Lepe,2283.0,63.0,97.0,27880.0,225.968436,347.919656,6.0
Lucena del Puerto,279.0,4.0,11.0,3261.0,122.661760,337.319841,0.0
Ayamonte,1432.0,35.0,65.0,21104.0,165.845337,307.998484,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Santa Bárbara de Casa,35.0,4.0,19.0,1043.0,383.509108,1821.668265,0.0,0.210526
Valverde del Camino,681.0,3.0,11.0,12750.0,23.529412,86.274510,1.0,0.272727
Alosno,207.0,5.0,18.0,3933.0,127.129418,457.665904,0.0,0.277778
Almonte,1956.0,11.0,36.0,24507.0,44.885135,146.896805,2.0,0.305556
Beas,235.0,1.0,3.0,4341.0,23.036167,69.108500,0.0,0.333333
Rosal de la Frontera,36.0,1.0,3.0,1697.0,58.927519,176.782557,0.0,0.333333
Lucena del Puerto,279.0,4.0,11.0,3261.0,122.661760,337.319841,0.0,0.363636
Bonares,319.0,5.0,13.0,6060.0,82.508251,214.521452,1.0,0.384615
Bollullos Par del Condado,1060.0,7.0,18.0,14387.0,48.655036,125.112949,1.0,0.388889
